In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob
from pandas.api.types import CategoricalDtype

In [2]:
import multiprocessing as mproc
NCPU = mproc.cpu_count()
print("Running on {} cores".format(NCPU))

Running on 2 cores


In [3]:
#if you upload a zip file to JupyterLab:
#import zipfile as zf
#files = zf.ZipFile("/home/jovyan/data.zip", 'r')
#files.extractall('/home/jovyan/')
#files.close()

In [4]:
# getting files from either Drive or locally
df = pd.DataFrame()
initial = "/content/drive/MyDrive/Colab Notebooks/" 
#initial = ""
#all_files = glob.glob(initial + "data/*")

# student data
course_data = glob.glob(initial + "data/uc.berkeley.student.course.data.*.txt")
majors = glob.glob(initial + "data/uc.berkeley.student.majors.*.csv")
pathways = glob.glob(initial + "data/uc.berkeley.stem.*.txt")

# jacobs data
jacobs = glob.glob(initial + "data/jacobs.participants.txt")
bcdi = glob.glob(initial + "data/jacobs.participants.bcdi.*")
waivers = glob.glob(initial + "data/jacobs.participants.waiver.*")
beeps = glob.glob(initial + "data/jacobs.participants.makerspace.*")

In [5]:
# Be aware, this cell is very resource intensive and takes some time to run
course_data.sort()
student_courses = pd.DataFrame()
#for f in course_data:
    #student_courses = student_courses.append(pd.read_table(f))
student_courses = pd.concat([pd.read_table(f) for f in course_data])
# minor data cleaning
student_courses = student_courses[~student_courses['ppsk'].isnull()]
student_courses = student_courses[student_courses['ppsk'] != -1]
for i in student_courses.columns:
    if student_courses[i].isnull().all():
        student_courses.drop(i,axis=1,inplace=True)
student_courses['ppsk'] = student_courses['ppsk'].astype(int)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
# every student who may have a maker pass
jacobs = pd.read_table(jacobs[0])
# may want to include missing ppsk's for full data fidelity later
#jacobs = jacobs[jacobs['ppsk'] != 'MISSING']

# stem pathways/ethnicity data
stem_paths = pd.read_table(pathways[0])
# students with waivers
waivers = pd.read_csv(waivers[0])

# all students who took design innovation course
jacobs_courses = student_courses[student_courses["course.subject"] == "Design Innovation"].drop('instructor.name', axis=1)

# student majors
student_majors = pd.DataFrame()
for i in majors:
    student_majors = student_majors.append(pd.read_csv(i))
student_majors.insert(2,'year.name', student_majors['year.majors'].astype(str) + " " + student_majors['term.majors'])
student_majors.drop(['year.majors', 'term.majors'], axis=1,inplace=True)
student_majors = student_majors.sort_values(by='year.name')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (33,43,44,50,51,53,55,56,57,58,59,60,61,62,63,64,65,66,67,72,73,74,75,76,77,78,79,80,81,82,83,89,90,92,93,94,95,96,97,98,99,100,101,102,103) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [7]:
# adding ethnicity data to jacobs students
jacobs_ethnicities = jacobs_courses.merge(stem_paths[['ppsk', 'ethnic.l1', 'gender']], on='ppsk', how='left')
jacobs_ethnicities['ethnic.l1'].fillna('Unknown', inplace=True)

# adding columns and created sortable year/term
jacobs_ethnicities = jacobs_ethnicities.rename({'ethnic.l1': "Ethnicity"}, axis=1)
jacobs_ethnicities['year'] = jacobs_ethnicities['year.name'].str.split(" ", expand=True)[0]
jacobs_ethnicities['term'] = jacobs_ethnicities['year.name'].str.split(" ", expand=True)[1]

# sorting and cleaning up dataframe
cat_size_order = CategoricalDtype(['Spring', 'Summer', 'Fall'], ordered=True)
jacobs_ethnicities['term'] = jacobs_ethnicities['term'].astype(cat_size_order)
jacobs_ethnicities = jacobs_ethnicities.sort_values(['year','term', 'Ethnicity'])
jacobs_ethnicities['year.name'] = jacobs_ethnicities['year'].astype(str) + " " + jacobs_ethnicities['term'].astype(str)
jacobs_ethnicities.drop(['year','term'], axis=1, inplace=True)

In [8]:
%store student_courses
%store student_majors
%store stem_paths
%store jacobs
%store jacobs_ethnicities
%store bcdi
%store waivers
%store beeps

Stored 'student_courses' (DataFrame)
Stored 'student_majors' (DataFrame)
Stored 'stem_paths' (DataFrame)
Stored 'jacobs' (DataFrame)
Stored 'jacobs_ethnicities' (DataFrame)
Stored 'bcdi' (list)
Stored 'waivers' (DataFrame)
Stored 'beeps' (list)


In [9]:
del student_courses
del student_majors
del stem_paths
del jacobs
del jacobs_ethnicities
del bcdi
del waivers
del beeps